# Tutorial
First, we need to import all the necessary modules.

In [ ]:
%load_ext autoreload
%autoreload 2

# built-in libraries
import os 

# third party libraries
import matplotlib.pyplot as plt 
import numpy as np
import tqdm
import torch 
import torch.nn.functional as func

# repository modules
import models
import train
import adversarial.carlini_wagner as cw
import adversarial.pgd as pgd
import dataset
import utils

The training and test data is loaded in memory using the `dataset` local module. Before using the FAUST dataset download it from [here](http://faust.is.tue.mpg.de/) and place it in `{repository-root}/datasets/faust/raw/`.

In [ ]:
REPO_ROOT = os.path.join(os.path.dirname(os.path.realpath('__file__')),"..")
FAUST = os.path.join(REPO_ROOT,"datasets/faust")
PARAMS_FILE = os.path.join(REPO_ROOT, "model_data/data.pt")

traindata = dataset.FaustDataset(FAUST, train=True, test=False, transform_data=True)
testdata = dataset.FaustDataset(FAUST, train=False, test=True,  transform_data=True)

Then we can define the classifier for the human pose classefication task; download the classifier parameters from [here](https://drive.google.com/open?id=1IllCPPPcqFL3v8BDxCyswzEpxoGyXvMe). Move these parameters in `{repository-root}/model_data/data.pt` .

In [ ]:
model = models.ChebnetClassifier(
    param_conv_layers=[128,128,64,64],
    D_t = traindata.downscale_matrices,
    E_t = traindata.downscaled_edges,
    num_classes = traindata.num_classes,
    parameters_file=PARAMS_FILE)

Training and evaluation can be done through the `train` local module.

In [ ]:
#train network
train.train(
    train_data=traindata,
    classifier=model,
    parameters_file=PARAMS_FILE,
    epoch_number=0)

#compute accuracy
accuracy, confusion_matrix = train.evaluate(eval_data=testdata, classifier=model)

print(accuracy)
plt.matshow(confusion_matrix)
plt.show()

In [ ]:
sum = 0
for mesh in tqdm.tqdm(traindata):
    i, j = mesh.edge_index
    xij = mesh.pos[i] - mesh.pos[j]
    xij_norm = xij.norm(p=2, dim=-1)
    v, _ = xij_norm.median(dim=-1)
    sum +=v
print(sum/len(traindata))
    

### Carlini & Wagner Method
It is possible to create adversarial examples through the **AdversarialExampleBuilder** and **AdversarialExample** classes  

In [ ]:
# get a random mesh from the dataset and a random target class
import random
i = random.randint(0, len(traindata)-1)
x = traindata[i].pos
e = traindata[i].edge_index.t() # needs to be transposed
f = traindata[i].face.t() # needs to be transposed
y = traindata[i].y
t = random.randint(0, traindata.num_classes-1)
eigs_num = 10

# configure adversarial example components
builder = cw.CWBuilder(search_iterations=1)
builder.set_classifier(model)
builder.set_perturbation_type("lowband", eigs_num=eigs_num)
builder.set_mesh(x, e, f)
builder.set_target(t)
builder.set_distortion_function(cw.L2_distortion)
builder.set_minimization_iterations(100).set_learning_rate(1e-4)
adex = builder.build(usetqdm="standard")
print("adversarial attack: "+("successful" if adex.is_successful else "unsuccessful"))


In [ ]:
visualize(adex.perturbed_pos, f, (x-adex.perturbed_pos).norm(p=2,dim=-1))
compare(adex.pos, f, adex.perturbed_pos,f)

We can finally see the actual adversarial example using **Plotly** (note: you need to install plotly before-hand)

In [ ]:
import plotly
import plotly.graph_objects as go
import numpy as np

def visualize(pos, faces, intensity=None):
  cpu = torch.device("cpu")
  if type(pos) != np.ndarray:
    pos = pos.to(cpu).clone().detach().numpy()
  if pos.shape[-1] != 3:
    raise ValueError("Vertices positions must have shape [n,3]")
  if type(faces) != np.ndarray:
    faces = faces.to(cpu).clone().detach().numpy()
  if faces.shape[-1] != 3:
    raise ValueError("Face indices must have shape [m,3]") 
  if intensity is None:
    intensity = np.ones([pos.shape[0]])
  elif type(intensity) != np.ndarray:
    intensity = intensity.to(cpu).clone().detach().numpy()

  x, z, y = pos.T
  i, j, k = faces.T

  mesh = go.Mesh3d(x=x, y=y, z=z,
            color='lightpink',
            intensity=intensity,
            opacity=1,
            colorscale=[[0, 'gold'],[0.5, 'mediumturquoise'],[1, 'magenta']],
            i=i, j=j, k=k,
            showscale=True)
  layout = go.Layout(scene=go.layout.Scene(aspectmode="data")) 

  #pio.renderers.default="plotly_mimetype"
  fig = go.Figure(data=[mesh],
                  layout=layout)
  fig.update_layout(
      autosize=True,
      margin=dict(l=20, r=20, t=20, b=20),
      paper_bgcolor="LightSteelBlue")
  fig.show()
    
def compare(pos1, faces1, pos2, faces2):
    n,m = pos1.shape[0], pos2.shape[0]
    tmpx = torch.cat([pos1, pos2],dim=0)
    tmpf = torch.cat([faces1, faces2+n], dim=0)
    color = torch.zeros([n+m],dtype=pos1.dtype, device=pos1.device)
    color[n:] = (pos1-pos2).norm(p=2,dim=-1)
    visualize(tmpx, tmpf,color)

### Projected Gradient Descent Method

In [128]:
from os import mkdir, listdir
from os.path import exists, join 
import random
import tqdm

import adversarial.pgd as pgd
import adversarial.carlini_wagner as cw
import numpy as np

def save_adex(adex, y, filename):
  ppos = adex.perturbed_pos.detach().clone().cpu().numpy()
  pos = adex.pos.detach().clone().cpu().numpy()
  edges = adex.edges.detach().clone().cpu().numpy()
  faces = adex.faces.detach().clone().cpu().numpy()
  is_successful = adex.is_successful
  
  data = {"perturbed-positions":ppos,
   "positions":pos,
   "edges":edges,
   "faces":faces,
    "y":y,
    "success":is_successful,
   "l2":cw.L2_distortion(adex).item(),
   "arap":cw.LocallyEuclideanDistortion(K=40)(adex).item()
   }
  np.save(filename, data, allow_pickle=True)

def pgd_experiments(projection_type, builder_type, model, data, folder="tmp"):
    projections = {"lowband":pgd.lowband_filter,
                  "lowband-clip":lambda a,x: pgd.clip(a, pgd.lowband_filter(a,x)),
                  "lowband-clipnorm":lambda a,x: pgd.clip_norms(a,pgd.lowband_filter(a,x)),
                   "none":lambda y,z : z,
                  "clip":pgd.clip,
                  "clipnorm":pgd.clip_norms}

    builders = {"lowband-filter":(pgd.LowbandPGDBuilder,1), "sign":(pgd.PGDBuilder,0.01), "l2":(pgd.L2PGDBuilder, 1)}
    successes, failures = 0,0
    
    if not exists(folder):
        mkdir(folder)
    
    for i in tqdm.tqdm(range(len(data))):
        x = data[i].pos
        e = data[i].edge_index.t()
        f = data[i].face.t() 
        y = data[i].y
            
        if model(x).argmax() == y:
            tmp, alpha = builders[builder_type]
            builder = tmp().set_iterations(7).set_epsilon(0.045).set_alpha(alpha).set_eigs_number(50)
            builder.set_projection(projections[projection_type])
            builder.set_mesh(x,e,f).set_classifier(model)
            adex = builder.build(usetqdm="standard")
            print("successful: {}".format(adex.is_successful))
            #print("adversarial example's prediction: {}".format(model(adex.perturbed_pos).argmax()))
            #print("ground-truth: {}".format(model(adex.pos).argmax()))
            #visualize(adex.pos, f, (adex.pos-adex.perturbed_pos).norm(dim=-1,p=2))
            #compare(adex.pos, f, adex.perturbed_pos, f)
            
            filename = join(folder, "adversarial_{}_b{}_p{}".format(i, builder_type, projection_type))
            save_adex(adex, y, filename)
            successes +=1
        else:
            print("skip")
            failures +=1
    return successes/(successes+failures)

pgd_experiments("lowband","l2", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_l2_lowband"))
pgd_experiments("none","l2", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_l2"))

pgd_experiments("lowband","sign", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_sign_lowband"))
pgd_experiments("none", "sign", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_sign"))
                                                                    
                                                                    
pgd_experiments("lowband-clipnorm","l2", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_l2_lowband-clipnorm"))
pgd_experiments("clipnorm","l2", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_l2_clipnorm"))
                                                                    
                                                                    
pgd_experiments("lowband-clipnorm","sign", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_sign_lowband-clipnorm"))
pgd_experiments("clipnorm","sign", model=model, data=testdata, folder=join(REPO_ROOT,"pgd_tests_sign_clipnorm"))

successful: False






100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [05:36<00:00, 16.82s/it]




  0%|                                                                                           | 0/20 [00:00<?, ?it/s]



  5%|████▏                                                                              | 1/20 [00:00<00:05,  3.69it/s]

skip
successful: False






 10%|████████▎                                                                          | 2/20 [00:18<01:40,  5.58s/it]

successful: False


...

successful: True






 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [05:33<00:19, 19.68s/it]

successful: False






100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [05:57<00:00, 17.86s/it]


0.9